# SysML → USD Digital Assembly Pipeline

This notebook demonstrates the full **S24 pipeline**:

1. Parse SysML v2 models (system + materials)
2. Convert to structured JSON representations
3. Vet and validate the system graph & material references
4. Generate a USD material library from SysML-defined properties
5. Generate USD geometry and components with material bindings
6. Assemble the full system hierarchy into a USD scene

### Architecture Overview

- **SysML** = Source of Truth (structure, hierarchy, materials identity + physical properties)
- **USD** = Integration layer (geometry + material look + physical data combined)
- **Omniverse** = Visualization & spatial exploration (later)

The output is a USD assembly suitable for visualization (usdview / Omniverse), simulation coupling, and digital twin workflows.

## Setup

In [2]:
import sys
from pathlib import Path

# Add repo root to Python path
ROOT = Path.cwd().parent
sys.path.insert(0, str(ROOT))

# Standard library
import os
import json

# Verify USD is available
from pxr import Usd, UsdShade
print("USD loaded successfully")

# S24 pipeline imports
from S24.sysml.exporter import sysml_to_json, write_json, sysml_to_materials, write_materials_json
from S24.jsonio.vetting import VettingProc
from S24.usd.builder import USDBuilder
from S24.usd.material_library import generate_material_library

USD loaded successfully


## Project Paths

We explicitly define all paths so the pipeline is reproducible and independent of the working directory.

In [3]:
# SysML sources
SYSML_DIR       = ROOT / "database" / "sysml"
SYSML_SYSTEM    = SYSML_DIR / "habitation.sysml"
SYSML_MATERIALS = SYSML_DIR / "materials.sysml"

# JSON outputs
JSON_DIR          = ROOT / "database" / "json"
JSON_PARTS        = JSON_DIR / "habmod.json"
JSON_MATERIALS    = JSON_DIR / "materials.json"

# USD outputs
ASSETS_DIR        = ROOT / "database" / "assets"
MAT_LIBRARY_PATH  = ASSETS_DIR / "mtl" / "lunar_materials.usda"
SCENES_DIR        = ROOT / "database" / "scenes"
SCENE_FILE        = SCENES_DIR / "HabitationAssembly.usda"

# Ensure directories exist
for d in [SYSML_DIR, JSON_DIR, ASSETS_DIR / "mtl", SCENES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("SysML system: ", SYSML_SYSTEM)
print("SysML materials:", SYSML_MATERIALS)
print("JSON parts:     ", JSON_PARTS)
print("JSON materials: ", JSON_MATERIALS)
print("Material library:", MAT_LIBRARY_PATH)
print("Scene output:   ", SCENE_FILE)

SysML system:  /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/sysml/habitation.sysml
SysML materials: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/sysml/materials.sysml
JSON parts:      /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/json/habmod.json
JSON materials:  /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/json/materials.json
Material library: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/assets/mtl/lunar_materials.usda
Scene output:    /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/scenes/HabitationAssembly.usda


## Step 1 — Load SysML Models

We load two SysML v2 models:
- **habitation.sysml**: system structure, hierarchy, dimensions, and `materialRef` identifiers
- **materials.sysml**: material definitions with physical properties (density, thermal conductivity, etc.)

SysML is the **source of truth** for all engineering data.

In [4]:
# Load system model
sysml_system_text = SYSML_SYSTEM.read_text(encoding="utf-8")
print("=== System Model (first 600 chars) ===")
print(sysml_system_text[:600], "...")

=== System Model (first 600 chars) ===
    package 'Habitation Module' {

        // -------------------------------------------------------------------------
        // Basic items
        // -------------------------------------------------------------------------
        private import SysML::*;
        private import SI::*;
        private import ISQBase::*;
        private import ScalarValues::*;

        part HabitationModule {
            part HabitationModule_dims {
                attribute length = 15; // meters
                attribute width = 10; // meters
                attribute height = 5; // meters

               ...


In [5]:
# Load materials model
sysml_materials_text = SYSML_MATERIALS.read_text(encoding="utf-8")
print("=== Materials Model (first 600 chars) ===")
print(sysml_materials_text[:600], "...")

=== Materials Model (first 600 chars) ===
package 'Materials' {

    private import SysML::*;
    private import SI::*;
    private import ISQBase::*;
    private import ScalarValues::*;

    // -------------------------------------------------------------------------
    // Material definition (source of truth for physical properties)
    // -------------------------------------------------------------------------
    part def Material {
        attribute materialId : String;

        // Physical properties (SI units expected)
        attribute density_kg_m3;
        attribute youngModulus_Pa;
        attribute poissonRatio;
         ...


## Step 2 — Export SysML to JSON

### 2a. System parts

The system model is converted into a flat JSON list of parts with:
- Explicit parent/children relationships
- Dimensions in SI units
- Evaluated attributes
- `materialRef` identifiers (stable IDs, not file paths)

In [6]:
parts_json = sysml_to_json(sysml_system_text, namespace="lunarspaceport1")
write_json(parts_json, str(JSON_PARTS))

print(f"Exported {len(parts_json)} parts to {JSON_PARTS.name}")
print()

# Show first part
print(json.dumps(parts_json[0], indent=2))

Exported 3 parts to habmod.json

{
  "type": "Part",
  "id": "urn:lunarspaceport1:part:HabitationModule:001",
  "name": "HabitationModule",
  "dimensions": {
    "dims_m": [
      15.0,
      10.0,
      5.0
    ],
    "metersPerUnit": 1,
    "upAxis": "Z",
    "X": 0,
    "Y": 0,
    "Z": 0
  },
  "attributes": {
    "HabitationModule_volume_m3": 750.0
  },
  "metadata": {
    "geometry": "assets/geom/HabModule_geom.usda"
  },
  "materialRef": "Titanium_Ti6Al4V",
  "children": [
    "O2Tank1",
    "O2Tank2"
  ]
}


### 2b. Material definitions

Material physical properties are exported from SysML into a dedicated `materials.json`.
These properties include density, Young's modulus, thermal conductivity, etc.

In [7]:
materials = sysml_to_materials(sysml_materials_text)
write_materials_json(materials, str(JSON_MATERIALS))

print(f"Exported {len(materials)} materials to {JSON_MATERIALS.name}")
print()

for mat in materials:
    print(f"  - {mat['materialId']}")
    for k, v in mat.items():
        if k != 'materialId':
            print(f"      {k}: {v}")

Exported 2 materials to materials.json

  - Aluminum_2219_T87
      density_kg_m3: 2840.0
      youngModulus_Pa: 73000000000.0
      poissonRatio: 0.33
      yieldStrength_Pa: 350000000.0
      ultimateStrength_Pa: 450000000.0
      thermalConductivity_W_mK: 120.0
      specificHeat_J_kgK: 900.0
      cte_1_K: 2.3e-05
      standard: AA 2219-T87
      source: TBD
  - Titanium_Ti6Al4V
      density_kg_m3: 4430.0
      youngModulus_Pa: 114000000000.0
      poissonRatio: 0.34
      yieldStrength_Pa: 880000000.0
      ultimateStrength_Pa: 950000000.0
      thermalConductivity_W_mK: 6.7
      specificHeat_J_kgK: 560.0
      cte_1_K: 8.6e-06
      standard: Ti-6Al-4V
      source: TBD


## Step 3 — Vet JSON Representation

We perform structural and semantic validation:
- Required fields present
- Unique part names
- Parent–child consistency (bidirectional)
- Cycle detection
- Single connected system graph

In [8]:
vetting = VettingProc(str(JSON_PARTS))
vetted_parts = vetting.by_name

print(f"Vetted {len(vetted_parts)} parts:")
for name, vp in vetted_parts.items():
    print(f"  - {name} (material: {vp.material_ref}, parent: {vp.parent})")

Vetted 3 parts:
  - HabitationModule (material: Titanium_Ti6Al4V, parent: None)
  - O2Tank1 (material: Aluminum_2219_T87, parent: HabitationModule)
  - O2Tank2 (material: Aluminum_2219_T87, parent: HabitationModule)


## Step 4 — Vet Material References

We verify that every `materialRef` used in a part actually exists in the material library.
This prevents broken bindings at the USD level.

In [9]:
known_material_ids = {m["materialId"] for m in materials}

print(f"Known materials: {sorted(known_material_ids)}")
print()

all_valid = True
for name, vp in vetted_parts.items():
    if vp.material_ref not in known_material_ids:
        print(f"  ERROR: Part '{name}' references unknown material '{vp.material_ref}'")
        all_valid = False
    else:
        print(f"  OK: {name} -> {vp.material_ref}")

if all_valid:
    print("\nAll material references are valid.")
else:
    raise ValueError("Material vetting failed — see errors above.")

Known materials: ['Aluminum_2219_T87', 'Titanium_Ti6Al4V']

  OK: HabitationModule -> Titanium_Ti6Al4V
  OK: O2Tank1 -> Aluminum_2219_T87
  OK: O2Tank2 -> Aluminum_2219_T87

All material references are valid.


## Step 5 — Generate USD Material Library

The material library is **auto-generated** from SysML-defined properties.
Each material becomes a `UsdShade.Material` prim with physical properties stored as `customData`.

No manual USD editing is needed — SysML is the single source of truth.

> **Note:** Visual shaders (PBR appearance) will be added later via Omniverse. For now, materials carry only physical data.

In [10]:
generate_material_library(str(JSON_MATERIALS), str(MAT_LIBRARY_PATH))

print(f"Generated material library: {MAT_LIBRARY_PATH}")
print()

# Show the generated USDA content
print(MAT_LIBRARY_PATH.read_text(encoding="utf-8"))

Generated material library: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/assets/mtl/lunar_materials.usda

#usda 1.0
(
    defaultPrim = "Materials"
)

def Scope "Materials"
{
    def Material "Aluminum_2219_T87" (
        customData = {
            double cte_1_K = 0.000023
            double density_kg_m3 = 2840
            double poissonRatio = 0.33
            string source = "TBD"
            double specificHeat_J_kgK = 900
            string standard = "AA 2219-T87"
            double thermalConductivity_W_mK = 120
            double ultimateStrength_Pa = 450000000
            double yieldStrength_Pa = 350000000
            double youngModulus_Pa = 73000000000
        }
    )
    {
    }

    def Material "Titanium_Ti6Al4V" (
        customData = {
            double cte_1_K = 0.0000086
            double density_kg_m3 = 4430
            double poissonRatio = 0.34
            string source = "TBD"
            double specificHeat_J_kgK = 560
            string

## Step 6 — Generate USD Assets

For each vetted part, we generate:
- **Geometry layer** (`*_geom.usda`) — placeholder box geometry
- **Component layer** (`<Part>.usda`) — references geometry, binds material from the shared library

Material binding uses `materialRef` to resolve against the generated `lunar_materials.usda`.

In [11]:
builder = USDBuilder(
    vetted_parts,
    database_dir=str(ROOT / "database"),
    overwrite=True,
    use_paths_from_vetted=False,
)

outputs = builder.build_all_parts()

print("Generated USD assets:")
for name, paths in outputs.items():
    print(f"  {name}:")
    for kind, path in paths.items():
        print(f"    {kind}: {path}")

Generated USD assets:
  HabitationModule:
    geom: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/assets/geoms/HabitationModule_geom.usda
    component: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/assets/components/HabitationModule.usda
  O2Tank1:
    geom: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/assets/geoms/O2Tank1_geom.usda
    component: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/assets/components/O2Tank1.usda
  O2Tank2:
    geom: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/assets/geoms/O2Tank2_geom.usda
    component: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/assets/components/O2Tank2.usda


## Step 7 — Assemble Full USD Scene

We instantiate the vetted hierarchy into a single USD stage
using component references and transforms.

The result is a complete system assembly under `/World/`.

In [12]:
scene_path = builder.write_assembly_scene(
    scene_name="HabitationAssembly.usda",
    root_name="HabitationModule",
    include_root_as_instance=True,
    instanceable=False,
    debug_refs=True,
)

print(f"Assembly scene written to: {scene_path}")

[REF] /World/HabitationModule -> ../assets/components/HabitationModule.usda :/HabitationModule
[REF] /World/HabitationModule/O2Tank1 -> ../assets/components/O2Tank1.usda :/O2Tank1
[REF] /World/HabitationModule/O2Tank2 -> ../assets/components/O2Tank2.usda :/O2Tank2
Assembly scene written to: /Users/arthur/Documents/GitHub/S24-ArchitectingLunarBases/database/scenes/HabitationAssembly.usda


## Step 8 — Inspect the Result

Let's verify the generated USD scene by opening it and inspecting the prim hierarchy.

In [13]:
stage = Usd.Stage.Open(scene_path)

print("USD Scene Hierarchy:")
for prim in stage.Traverse():
    depth = len(str(prim.GetPath()).split("/")) - 1
    indent = "  " * depth
    print(f"{indent}{prim.GetName()} ({prim.GetTypeName()})")

USD Scene Hierarchy:
  World (Xform)
    HabitationModule (Xform)
      geom (Mesh)
      materials (Scope)
        Aluminum_2219_T87 (Material)
        Titanium_Ti6Al4V (Material)
      O2Tank1 (Xform)
        geom (Mesh)
        materials (Scope)
          Aluminum_2219_T87 (Material)
          Titanium_Ti6Al4V (Material)
      O2Tank2 (Xform)
        geom (Mesh)
        materials (Scope)
          Aluminum_2219_T87 (Material)
          Titanium_Ti6Al4V (Material)


In [14]:
# Verify material bindings
print("Material Bindings:")
for prim in stage.Traverse():
    binding_api = UsdShade.MaterialBindingAPI(prim)
    mat, _ = binding_api.ComputeBoundMaterial()
    if mat:
        print(f"  {prim.GetPath()} -> {mat.GetPath()}")

Material Bindings:


## Summary

The pipeline successfully:

1. Parsed two SysML v2 models (system structure + material definitions)
2. Exported structured JSON with `materialRef` identifiers
3. Vetted the system graph and material references
4. Auto-generated a USD material library with physical properties from SysML
5. Built USD components with proper material bindings
6. Assembled a complete USD scene

### Architecture Principles

| Concern | Source of Truth | Tool |
|---|---|---|
| System structure & hierarchy | SysML | Parser + Exporter |
| Material identity & physical properties | SysML | Material library generator |
| Material visual appearance (shaders) | Omniverse (TODO) | Manual authoring |
| CAD geometry | STEP → USD via Omniverse (TODO) | Omniverse import |
| Integration & assembly | USD | S24 pipeline |

### View the result

```bash
usdview database/scenes/HabitationAssembly.usda
```

What follow is just a verification that all modules possess the correct material

In [16]:
# Check material data is written in component files
import os
from pxr import Usd

comps_dir = ROOT / "database" / "assets" / "components"
for f in sorted(os.listdir(comps_dir)):
    if f.endswith(".usda"):
        comp_stage = Usd.Stage.Open(str(comps_dir / f))
        print(f"=== {f} ===")
        for prim in comp_stage.Traverse():
            # Check material binding hint
            hint = prim.GetCustomDataByKey("materialHint")
            if hint:
                print(f"  {prim.GetName()}: materialHint = {hint}")
            # Check customData on material prims
            if prim.GetTypeName() == "Material":
                cd = prim.GetCustomData()
                if cd:
                    print(f"  {prim.GetName()}: customData = {dict(cd)}")
        print()

# Also check the material library directly
print("=== Material Library ===")
mat_stage = Usd.Stage.Open(str(MAT_LIBRARY_PATH))
for prim in mat_stage.Traverse():
    if prim.GetTypeName() == "Material":
        cd = prim.GetCustomData()
        print(f"  {prim.GetName()}: {dict(cd)}")

=== HabitationModule.usda ===
  geom: materialHint = ../mtl/lunar_materials.usda:/Materials/Titanium_Ti6Al4V
  Aluminum_2219_T87: customData = {'cte_1_K': 2.3e-05, 'density_kg_m3': 2840.0, 'poissonRatio': 0.33, 'source': 'TBD', 'specificHeat_J_kgK': 900.0, 'standard': 'AA 2219-T87', 'thermalConductivity_W_mK': 120.0, 'ultimateStrength_Pa': 450000000.0, 'userDocBrief': 'A Material provides a container into which multiple "render contexts"\n    can add data that defines a "shading material" for a renderer.', 'yieldStrength_Pa': 350000000.0, 'youngModulus_Pa': 73000000000.0}
  Titanium_Ti6Al4V: customData = {'cte_1_K': 8.6e-06, 'density_kg_m3': 4430.0, 'poissonRatio': 0.34, 'source': 'TBD', 'specificHeat_J_kgK': 560.0, 'standard': 'Ti-6Al-4V', 'thermalConductivity_W_mK': 6.7, 'ultimateStrength_Pa': 950000000.0, 'userDocBrief': 'A Material provides a container into which multiple "render contexts"\n    can add data that defines a "shading material" for a renderer.', 'yieldStrength_Pa': 880